Psycho-physiological research has shown that left and right frontal lobes have significant activity during the experience of emotions. The signals were selected from four positions Fp1(ch 1), Fp2(ch 17), F3(ch 3) and F4(ch20) only

In [3]:
import pickle
import os
from matplotlib import pyplot as plt 
import numpy as np
import mne
import seaborn as sns
from scipy.integrate import simps
import pandas as pd
from scipy import signal

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import svm, datasets
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [4]:
%matplotlib inline

In [5]:
os.chdir(r"C:\Users\Diksha Menghmalani\Desktop\BE Project\Deap dataset\data_preprocessed_python") 

In [6]:
channels = [0,16,2,19]
#channels = [0,2]
#channels = [0,1,2,3,5,10,12,16,18,19,20,24,28,30]
#channels = [i for i in range(32)]
t_low , t_high = 4,  7
a_low , a_high = 8,  13
b_low , b_high = 13, 30
g_low , g_high = 30, 40
columns_feature_video = ['fp1_abs_t','fp1_abs_a','fp1_abs_b','fp1_abs_g',
                          'fp1_mean_t','fp1_mean_a','fp1_mean_b','fp1_mean_g',
                         'fp2_abs_t','fp2_abs_a','fp2_abs_b','fp2_abs_g',
                          'fp2_mean_t','fp2_mean_a','fp2_mean_b','fp2_mean_g',
                         'f3_abs_t','f3_abs_a','f3_abs_b','f3_abs_g',
                          'f3_mean_t','f3_mean_a','f3_mean_b','f3_mean_g'
                         'f4_abs_t','f4_abs_a','f4_abs_b','f4_abs_g',
                          'f4_mean_t','f4_mean_a','f4_mean_b','f4_mean_g']

In [7]:
subjects = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16',
            '17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32']

In [8]:
for sub in subjects:
    with open('s' + sub + '.dat', 'rb') as file:
        
        subject = pickle.load(file, encoding='latin1')
        
        data = np.array(subject['data'])
        l = np.array(subject['labels'])
        
        for j in range(40):
            feature_video = []
        
            for i in channels:
                freqs, psd = signal.welch(data[0][i], fs= 128)
        
                idx_theta = np.logical_and(freqs >= t_low, freqs <= t_high)
                idx_alpha = np.logical_and(freqs >= a_low, freqs <= a_high)
                idx_beta = np.logical_and(freqs >= b_low, freqs <= b_high)
                idx_gamma = np.logical_and(freqs >= g_low, freqs <= g_high)
        
                #Absolute Power
                freq_res = freqs[1] - freqs[0]  
                theta_power = simps(psd[idx_theta], dx=freq_res)
                alpha_power = simps(psd[idx_alpha], dx=freq_res)
                beta_power = simps(psd[idx_beta], dx=freq_res)
                gamma_power = simps(psd[idx_gamma], dx=freq_res)
        
                #Mean Power
        
                t_mean = np.mean(psd[idx_theta])
                a_mean = np.mean(psd[idx_alpha])
                b_mean = np.mean(psd[idx_beta])
                g_mean = np.mean(psd[idx_gamma])
                
                #Std Dev Power
                
                t_std = np.std(psd[idx_theta])
                a_std = np.std(psd[idx_alpha])
                b_std = np.std(psd[idx_beta])
                g_std = np.std(psd[idx_gamma])
                
                
                # theta_power,alpha_power,beta_power,gamma_power,t_std,a_std,b_std,g_std
                f = [t_mean,a_mean,b_mean,g_mean]
                
                feature_video = np.array([np.append(feature_video,f)])
                #psd_all_channels = np.array([np.append(psd_all_channels,psd)])
                #print(psd_all_channels.shape)
                
            
            if sub=='01' and j ==0:
                features = feature_video
                
            else:
                features = np.append(features,feature_video,axis=0)
                
        # 0 : Happy; 1: Relaxed ; 2: Angry; 3: Sad
                
        labels_val_video = np.array([-1 for i in range(40)])
        labels_arousal_video = np.array([-1 for i in range(40)])
        '''
        for i in range(len(l)):
            if l[i][0]>=5 and l[i][1]>=5:
                labels_video[i] = 0
            elif l[i][0]>=5 and l[i][1]<=5:
                labels_video[i] = 1
            elif l[i][0]<=5 and l[i][1]>=5:
                labels_video[i] = 2
            elif l[i][0]<=5 and l[i][1]<=5:
                labels_video[i] = 3
        '''
        for i in range(len(l)):
            if l[i][0]<=4.5:
                labels_val_video[i] = 0
            
            #elif l[i][0]>=4 and l[i][0]<6 :
             #   labels_val_video[i] = 1
                
            elif l[i][0]>= 4.5:
                labels_val_video[i] = 2
                
        for i in range(len(l)):
            if l[i][1]<=4.5:
                labels_arousal_video[i] = 0
        
            #elif l[i][1]>=4 and l[i][1]<6 :
             #   labels_val_video[i] = 1
                
            elif l[i][1]>=4.5 :
                labels_val_video[i] = 2
                
        if sub == '01':
            labels_val = labels_val_video
            labels_arousal = labels_arousal_video
        else:
            labels_val = np.append(labels_val,labels_val_video)
            labels_arousal = np.append(labels_arousal,labels_arousal_video)
    

In [9]:
features.shape

(1280, 16)

In [10]:
labels_val.shape

(1280,)

In [11]:
labels_arousal.shape

(1280,)

df = pd.DataFrame(features)

df.head()

# Valence

In [12]:
X = features
y = labels_val

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size= 0.2,random_state= 4)

In [14]:
X_train.shape

(1024, 16)

In [15]:
y_train.shape

(1024,)

In [16]:
poly = svm.SVC(kernel='poly', degree=5, C=1, decision_function_shape='ovo').fit(X_train, y_train)

poly_pred = poly.predict(X_test)

accuracy_poly = poly.score(X_test, y_test)
print("Accuracy for SVM Classifier:%.3f "% (accuracy_poly*100))

Accuracy for SVM Classifier:87.500 


In [17]:

knn_clf=KNeighborsClassifier()
knn_clf.fit(X_train,y_train)

ypred=knn_clf.predict(X_test)


result = accuracy_score(y_test,ypred)
print("Accuracy for KNN Classifier: %.3f"%(result*100))

Accuracy for KNN Classifier: 83.984


In [18]:
#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)


# Model Accuracy
print('Accuracy of Random Forest Classifier: %.3f' %(metrics.accuracy_score(y_test, y_pred)*100))

Accuracy of Random Forest Classifier: 87.500


# AROUSAL

In [19]:
X = features
y = labels_arousal

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size= 0.2,random_state= 4)

In [25]:
poly = svm.SVC(kernel='poly', C=1).fit(X_train, y_train)

poly_pred = poly.predict(X_test)

accuracy_poly = poly.score(X_test, y_test)
print("Accuracy for SVM Classifier:%.3f "% (accuracy_poly*100))

Accuracy for SVM Classifier:65.234 


In [26]:

knn_clf=KNeighborsClassifier()
knn_clf.fit(X_train,y_train)

ypred=knn_clf.predict(X_test)

result = accuracy_score(y_test,ypred)
print("Accuracy for KNN Classifier: %.3f"%(result*100))

Accuracy for KNN Classifier: 63.672


In [27]:

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)


# Model Accuracy
print('Accuracy of Random Forest Classifier: %.3f' %(metrics.accuracy_score(y_test, y_pred)*100))

Accuracy of Random Forest Classifier: 62.500
